# Setup
Utilizaremos a biblioteca `boto3` para nos comunicarmos com a AWS.

Utilizaremos as bibliotecas `pandas` e `pyspark` para lidarmos com nossos dados.

Utilizaremos as bibliotecas `matplotlib` e `seaborn` para a análise final.

In [1]:
import pandas as pd
import boto3

In [2]:
BUCKET = '896309849144'
LAYERS_PREFIX = f'layers'# f's3://{BUCKET}/layers'
GOLD = f'{LAYERS_PREFIX}/gold'
SILVER = f'{LAYERS_PREFIX}/silver'
BRONZE = f'{LAYERS_PREFIX}/bronze'

In [3]:
s3_client = boto3.client('s3')

def upload_to_s3(
        file_path : str,
        bucket : str,
        bucket_path : str):
    '''
    Uploads file_path to the given bucket, as bucket_path (include the name of the file in bucket_path)
    
    Parameters
    -
    file_path : str
    bucket : str
    bucket_path : str
    '''
    s3_client.upload_file(file_path, bucket, bucket_path)

# 0. Ingestion
Faremos o upload dos dados brutos extraídos manualmente do IBGE, a um bucket AWS S3 via a biblioteca `boto3`.

Conforme o pedido, utilizaremos os últimos 3 meses disponíveis.

In [ ]:
LOCAL_BRONZE = './bronze'
files = [
    f'{LOCAL_BRONZE}/PNAD_COVID_092020.csv',
    f'{LOCAL_BRONZE}/PNAD_COVID_102020.csv',
    f'{LOCAL_BRONZE}/PNAD_COVID_112020.csv']

In [9]:
for file in files:
    print(file.split('/')[-1])

PNAD_COVID_092020.csv
PNAD_COVID_102020.csv
PNAD_COVID_112020.csv


In [10]:
for file in files:
    upload_to_s3(
        file,
        BUCKET,
        f'{BRONZE}/{file.split("/")[-1]}')

# 1. Bronze
O processo de re-estruturamento dos dados é feito na AWS via `AWS Glue`.

# 2. Silver
Pegaremos esses dados re-estruturados *silver* e faremos nossas queries via `PySpark`.

In [43]:
to_download = s3_client.list_objects_v2(
    Bucket = BUCKET,
    Prefix = SILVER
)['Contents'][1:]

count = 1

LOCAL_SILVER = './local/silver'

for file in to_download:
    s3_client.download_file(BUCKET, file['Key'], f'{LOCAL_SILVER}/silver_{count}.parquet')
    count+=1

## Checando integridade dos dados

Comparemos o tamanho da camade bronze original com a camada silver atual para nos certificarmos que o processo de merge ocorreu corretamente

In [47]:
df_silver = pd.read_parquet('./local/')
df_silver.shape[0]

1149197

In [46]:
row_count = 0
for f in files[:]:
    _ = pd.read_csv(f)
    row_count += _.shape[0]

    print(_.shape[0])
print(f'TOTAL = {row_count}')

387298
380461
381438
TOTAL = 1149197


## Selecionamento das perguntas
Foi requesitado a utilização de no máximo 20 perguntas da pesquisa. Seguem as selecionadas abaixo.